# CMPS 140

## Assignment 2

**DUE: Sunday July 7, 2019 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Kernel→→Restart) and then run all cells (in the menubar, select Cell→→Run All).
#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also show *how* you arrived at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
NAME = "Derrick DeBose"

## Adversarial Search

Imagine a non-stochastic game like tictactoe, checkers, chess, or go, where players alternate moves and have opposing goals. Prior to deep learning, one of the best approaches to writing sofware to play such games involved variations of the famous minimax algorithm https://en.wikipedia.org/wiki/Minimax. 

In this assignment, you'll be implementing the depth limited minimax algorithm, as well as it's more sophisticated form involving alpha-beta pruning https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning, and comparing the run times of these two algorithms.

## Minimax

A simple animation illustrating the process: https://www.youtube.com/watch?v=zDskcx8FStA 
In your implementation, you'll assume the nodes of the tree you're searching don't initially have values (they are all initialized to 0), except for the leaf nodes. Each layer of the tree represents the move options of one of two players, the Maximizer (assume this player is the first to play) who wishes to maximize her score, and the Minimizer, who hopes to minimize his score. In real-world applications one often has to create heuristics to approximate values for the leaf nodes, but in this assignmemt the values are already present in the leaf nodes. 


## Minimax Pseudocode

Below is pseudocode for the depth limited minimax algorithm. You'll need to code this up in Python.

In [ ]:
#Commented out psudocode to make sure there is no error
"""
*** Pseudocode for the depth limited minimax algorithm ***

function minimax(node, depth, maximizingPlayer) is

    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, minimax(child, depth − 1, FALSE))
        return value
    
    else (* minimizing player *)
        value := +∞
        for each child of node do
            value := min(value, minimax(child, depth − 1, TRUE))
        return value

    
(* Initial call *)
minimax(origin, depth, TRUE)
"""

'\n*** Pseudocode for the depth limited minimax algorithm ***\n\nfunction minimax(node, depth, maximizingPlayer) is\n\n    if depth = 0 or node is a terminal node then\n        return the heuristic value of node\n    \n    if maximizingPlayer then\n        value := −∞\n        for each child of node do\n            value := max(value, minimax(child, depth − 1, FALSE))\n        return value\n    \n    else (* minimizing player *)\n        value := +∞\n        for each child of node do\n            value := min(value, minimax(child, depth − 1, TRUE))\n        return value\n\n    \n(* Initial call *)\nminimax(origin, depth, TRUE)\n'

## The Input Tree

As a simple example of an input tree, imagine the following figure without numbers except in the leaf nodes.

![minimax](https://drive.google.com/uc?id=1klN5Jxli9S_cpfGsjTWumtNkt8A0nvd1)

On the left is the input tree, with root node 1 and 6 other consecutively numbered nodes. The leaf nodes have the values in boxes at the bottom of the tree. Your minimax algorithm will fill in the values missing in the input tree. Notice the optimal path in red, which involves the nodes 

$$[ 1, 3, 6 ]$$  

The above node list is the desired output from your minimax function for this input tree.

The entire tree can be recovered from its root node.  Each node in the tree has the following fields:

    ID = ID
    value = 0
    parent = parent_node 
    children = [None for _ in range(branching)]

The IDs are the node numbers. All values are initially 0, except for the leaf nodes. All nodes have a field containing a list of its children; the leaf nodes "children" are all None.

In [ ]:
# Useful library and functions

import numpy as np
from copy import deepcopy
import pickle


class node:
    def __init__(self, ID, parent_node, branching = 2):
        self.ID = ID
        self.value = 0
        self.parent = parent_node 
        self.children = [None for _ in range(branching)]

        
# Determine the depth of this tree.
def depth_of_tree(root):
    ''' This function assumes that all leaves are at the same depth '''
    depth = 0
    children = deepcopy(root.children)
    while children[0] != None:
        depth += 1
        children = deepcopy(children[0].children)
    return depth


# This function is useful to print out your tree node IDs with their values, for each level of the tree.
def print_tree(root):
    ''' Prints successive levels of the tree '''
    depth = depth_of_tree(root)
    print((root.ID, root.value))
    newchildren = deepcopy(root.children)
    
    for i in range(depth):
        children = deepcopy(newchildren)
        newchildren = []
        
        for c in children:
            print((c.ID, c.value), end=' ')
        print()
        
        for c in children:
            newchildren += c.children
            

# With this function, you'll be able to extract the optimal minimax path through  
# the tree from the root node to a leaf.
def minimax_path(root):
    ''' Assumes minimax has been run on the tree '''
    node = deepcopy(root)
    nodes = [node.ID]
    l = True
    
    while node.children[0] != None:
    
        vals = [c.value for c in node.children]
        if l:
            v = max(vals)
            i = np.argmax(vals)
        else:
            v = min(vals)
            i = np.argmin(vals)
            
        nodes.append(node.children[i].ID)
        node = node.children[i]
        l = not l
        
    return nodes

## Problem 1

Implement the above minimax pseudocode in Python.

In [ ]:
# The maximizingPlayer is True at the root node.
def minimax(node, depth, maximizingPlayer): 
    
    if depth == 0:
        return node.value
    #check most right child first because we have to check each child no matter what
    length = len(node.children)-1
    if maximizingPlayer:
        value = -np.inf
        #this loop ensures there is no index out of bounds and never call minimax on a leaf node
        while length != -1 and node.children[length] != None:
            value= max(value, minimax(node.children[length], depth-1, False))
            length -=1
        node.value = value
        return value
    else:
        value = np.inf
        #this loop ensures there is no index out of bounds and never call minimax on a leaf node
        while length != -1 and node.children[length] != None:
            value= min(value, minimax(node.children[length], depth-1, True))
            length -=1
        node.value = value
        return value

# Test cases

The code below takes as input the output from your minimax function, and it ouputs the optimal path according to your algorithm.

Your submitted code will be run on somewhat larger input trees.

In [21]:
#################
## Test Case 1 ##
#################

# IMPORTANT #

# If you need this, remember to first put the files you want to read in onto your Google Drive.
# See Piazza for more details, @115.
#
#from google.colab import files
#uploaded = files.upload()

# Read in the tree.
tree = pickle.load(open('tree_3_2.pkl', 'rb' ))

# You can see what's in this tree:
print_tree(tree);  print()

# Call your minimax function, which returns the value of the game that still needs to be stored in the root node:
depth = 3
tree.value = minimax(tree, depth, True)

if minimax_path(tree) == [1, 3, 6, 13]:
    print('Test 1 passed')
else:
    print('Test 1 failed')

    
# If you print out the tree again (after implementing minimax) those 0s will be filled in 
# with the correct values for each node
#print_tree(tree)

FileNotFoundError: ignored

In [ ]:
#################
## Test Case 2 ##
#################

# Read in the tree.
tree = pickle.load(open('tree_3_3.pkl', 'rb' ))
minimax_path(tree)

# Call your minimax function, which returns the value of the game that still needs to be stored in the root node:
depth = 3
tree.value = minimax(tree, depth, True)

if minimax_path(tree) == [1, 4, 13, 41]:
    print('Test 2 passed')
else:
    print('Test 2 failed')
    

#################
## Test Case 3 ##
#################

# Read in the tree.
tree = pickle.load(open('tree_5_3.pkl', 'rb' ))

# Call your minminimax_path(tree)imax function, which returns the value of the game that still needs to be stored in the root node:
depth = 5
tree.value = minimax(tree, depth, True)

if minimax_path(tree) == [1, 4, 13, 39, 117, 352]:
    print('Test 3 passed')
else:
    print('Test 3 failed')

## Alpha-Beta Pruning

A video describing this algorithm: https://www.youtube.com/watch?v=l-hh51ncgDI 

This is quite similar to your above minimax algorithm, but with some pruning to speed things up.

## Alpha-Beta Pruning Pseudocode

Below is pseudocode for the depth limited alpha-beta pruning algorithm, from the Wikipedia article linked to above. You'll need to code this up in Python.

In [ ]:
#Commented out psudocode to make sure there is no error
"""
*** Pseudocode for the depth limited alpha-beta pruning algorithm ***

function alphabeta(node, depth, α, β, maximizingPlayer) is

    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, alphabeta(child, depth − 1, α, β, FALSE))
            α := max(α, value)
            if α ≥ β then
                break (* β cut-off *)
        return value
    
    else
        value := +∞
        for each child of node do
            value := min(value, alphabeta(child, depth − 1, α, β, TRUE))
            β := min(β, value)
            if α ≥ β then
                break (* α cut-off *)
        return value
    
(* Initial call *)
alphabeta(origin, depth, −∞, +∞, TRUE)
"""

## Problem 2

Implement the above depth limited alpha-beta pruning pseudocode in Python.

In [ ]:
# The maximizingPlayer is True at the root node.
def alphabeta(node, depth, alpha, beta, maximizingPlayer):
    if depth == 0:
        return node.value
    length = len(node.children)
    #leng is used as a loop counter to make sure we prune the right nodes in that level
    leng = 0
    if maximizingPlayer:
        value = -np.inf
        #this loop ensures there is no index out of bounds and never call minimax on a leaf node
        while leng != length and node.children[leng] != None:
            value= max(value, alphabeta(node.children[leng], depth-1, alpha, beta, False))
            alpha= max(alpha, value)
            leng +=1
            if alpha >= beta:
                break
    else:
        value = np.inf
        #this loop ensures there is no index out of bounds and never call minimax on a leaf node
        while leng != length and node.children[leng] != None:
            value= min(value, alphabeta(node.children[leng], depth-1, alpha, beta, True))
            beta= min(beta, value)
            leng +=1
            if alpha >= beta:
                break
    
    node.value = value
    return value

### Test cases

The code below takes as input the output from your alpha-beta pruning function, and it ouputs the optimal path according to your algorithm. You should get the same optimal paths as with your minimax algoritym. 

Your submitted code will be run on somewhat larger input trees.

In [ ]:
################
## Test Cases ##
################

# If you need this, remember to first put the files you want to read in onto your Google Drive.
# See Piazza for more details, @115.
#
#from google.colab import files
#uploaded = files.upload()

# Test Case 1

# Read in the tree.
tree = pickle.load(open('tree_3_2.pkl', 'rb' ))

# Call your minimax function, which returns the value of the game that still needs to be stored in the root node:
depth = 3
tree.value = alphabeta(tree, depth, -np.inf, np.inf, True)

if minimax_path(tree) == [1, 3, 6, 13]:
    print('Test 1 passed')
else:
    print('Test 1 failed')
    
    
# Test Case 2

# Read in the tree.
tree = pickle.load(open('tree_3_3.pkl', 'rb' ))
minimax_path(tree)

# Call your minimax function, which returns the value of the game that still needs to be stored in the root node:
depth = 3
tree.value = alphabeta(tree, depth, -np.inf, np.inf, True)

if minimax_path(tree) == [1, 4, 13, 41]:
    print('Test 2 passed')
else:
    print('Test 2 failed')
    

# Test Case 3

# Read in the tree.
tree = pickle.load(open('tree_5_3.pkl', 'rb' ))

# Call your minminimax_path(tree)imax function, which returns the value of the game that still needs to be stored in the root node:
depth = 5
tree.value = alphabeta(tree, depth, -np.inf, np.inf, True)

if minimax_path(tree) == [1, 4, 13, 39, 117, 352]:
    print('Test 3 passed')
else:
    print('Test 3 failed')

## Problem 3

Next you'll compare the running times of your two algorithms; you'll find that alpha-beta pruning is much faster. 

In [ ]:
from time import time

depth = 15

# Read in the tree.
tree = pickle.load(open('tree_15_2.pkl', 'rb' ))

start_time = time()
tree.value = minimax(tree, depth, True)
time1 = time() - start_time
# Read in the tree.
tree = pickle.load(open('tree_15_2.pkl', 'rb' ))

start_time = time()
tree.value = alphabeta(tree, depth, -np.inf, np.inf, True)
time2 = time() - start_time
print('path =', minimax_path(tree))
print(time1/time2)

3(a)  What is the optimal path you found?

#### path = [1, 2, 4, 9, 18, 36, 72, 144, 288, 576, 1153, 2307, 4614, 9229, 18458, 36917]

3(b)  How many times faster was your alphabeta pruning function than your minimax function?

#### 7.629024639848855

In [ ]:
from time import time

depth = 10

# Read in the tree.
tree = pickle.load(open('tree_10_3.pkl', 'rb' ))

start_time = time()
tree.value = minimax(tree, depth, True)
time1 = time() - start_time
# Read in the tree.
tree = pickle.load(open('tree_10_3.pkl', 'rb' ))

start_time = time()
tree.value = alphabeta(tree, depth, -np.inf, np.inf, True)
time2 = time() - start_time

print('path =', minimax_path(tree))
print(time1/time2)

3(c)  What is the optimal path you found?

#### path = [1, 3, 11, 33, 99, 299, 897, 2692, 8077, 24231, 72694]

3(d)  How many times faster was your alphabeta pruning function than your minimax function?

#### 8.974873592455873